In [13]:
#Hardware accelerator: GPU
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pwd
%cd /content/drive/MyDrive/kaggle

/content/drive/My Drive/kaggle
/content/drive/MyDrive/kaggle


In [15]:
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [16]:
# Setup data inputs
IMG_SIZE = (224, 224)

def create_data_loaders(train_dir, test_dir, image_size=IMG_SIZE):
  train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)

  test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  return train_data, test_data

In [17]:
# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [18]:
# Setup input shape and base model, freezing the base model layers
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
  base_model.trainable = False

  # Create input layer
  inputs = layers.Input(shape=input_shape, name="input_layer")

  # Add in data augmentation Sequential model as a layer
  x = data_augmentation(inputs)

  # Give base_model inputs (after augmentation) and don't train it
  x = base_model(x, training=False)

  # Pool output features of base model
  x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

  # Put a dense layer on as the output
  outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

  # Make a model with inputs and outputs
  model = keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  
  return model

16711680/16705208 [==============================] - 0s 0us/step


In [19]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=False):
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  if scale:
    return img/255.
  else:
    return img

In [20]:
# Unzip the downloaded file
def unzip_data(filename):
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [21]:
# Get data
import zipfile
unzip_data("10_food_classes_all_data.zip")

In [22]:
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

## Model 1 (10 classes)

In [23]:
# Create BatchDataset
train_data, test_data = create_data_loaders(train_dir="10_food_classes_all_data/train/",
                                            test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [25]:
train_data

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [26]:
# Create model
model_1 = create_model(num_classes=len(train_data.class_names))

# Fit the model
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)), # validate for less steps
                    # Track model training logs
                    callbacks=[create_tensorboard_callback("transfer_learning", "all_data_aug")])

Saving TensorBoard log files to: transfer_learning/all_data_aug/20210427-144442
Epoch 1/5
235/235 [==============================] - 172s 581ms/step - loss: 1.4528 - accuracy: 0.5444 - val_loss: 0.5686 - val_accuracy: 0.8421
Epoch 2/5
235/235 [==============================] - 108s 455ms/step - loss: 0.7062 - accuracy: 0.7863 - val_loss: 0.4615 - val_accuracy: 0.8651
Epoch 3/5
235/235 [==============================] - 101s 426ms/step - loss: 0.6157 - accuracy: 0.8110 - val_loss: 0.4649 - val_accuracy: 0.8454
Epoch 4/5
235/235 [==============================] - 95s 402ms/step - loss: 0.5625 - accuracy: 0.8256 - val_loss: 0.4580 - val_accuracy: 0.8520
Epoch 5/5
235/235 [==============================] - 90s 381ms/step - loss: 0.5257 - accuracy: 0.8375 - val_loss: 0.4449 - val_accuracy: 0.8635


In [27]:
# Get an image Tensor
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg

--2021-04-27 15:28:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874848 (2.7M) [image/jpeg]
Saving to: ‘03-pizza-dad.jpeg’

03-pizza-dad.jpeg   100%[===================>]   2.74M  --.-KB/s    in 0.1s    

2021-04-27 15:28:01 (24.1 MB/s) - ‘03-pizza-dad.jpeg’ saved [2874848/2874848]



In [28]:
# Classes our model is trained on
class_names = train_data.class_names
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [29]:
# Preprocess image
pizza_img = load_and_prep_image("03-pizza-dad.jpeg")
pizza_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 73.625,  76.75 ,  67.125],
        [114.   , 122.   , 101.   ],
        [146.875, 151.875, 129.875],
        ...,
        [ 14.5  ,  17.5  ,  10.5  ],
        [ 14.25 ,  19.25 ,  12.25 ],
        [ 19.75 ,  22.75 ,  15.75 ]],

       [[239.125, 243.625, 246.125],
        [225.375, 232.125, 234.875],
        [240.   , 245.   , 244.5  ],
        ...,
        [ 11.   ,  14.   ,   7.   ],
        [ 20.   ,  23.   ,  16.   ],
        [ 20.875,  25.875,  18.875]],

       [[ 32.5  ,  34.5  ,  31.5  ],
        [ 44.625,  44.5  ,  42.375],
        [ 33.   ,  38.   ,  34.   ],
        ...,
        [  8.75 ,  13.25 ,   6.25 ],
        [ 14.875,  17.875,  10.875],
        [ 13.625,  20.625,  12.625]],

       ...,

       [[ 61.875,  40.875,  19.875],
        [ 60.   ,  42.   ,  22.   ],
        [ 61.   ,  43.   ,  21.   ],
        ...,
        [134.5  ,  96.125,  60.75 ],
        [104.875,  69.375,  43.125],
        [106.25 ,  75.2

In [30]:
# Make predictions
pizza_expanded = tf.expand_dims(pizza_img, axis=0) # expand image dimensions (224, 224, 3) -> (1, 224, 224, 3)
pred = model_1.predict(pizza_expanded)
pred

array([[1.6281351e-02, 5.3865446e-05, 8.9909328e-04, 4.0310339e-04,
        5.2360929e-06, 1.0736467e-06, 9.8034036e-01, 6.3052826e-04,
        2.6637688e-04, 1.1190677e-03]], dtype=float32)

In [31]:
# Check the predicted class
class_names[tf.argmax(pred[0])]

'pizza'

In [32]:
# Save model_1
model_1.save("efficientnet_model_1_10_classes")

INFO:tensorflow:Assets written to: efficientnet_model_1_10_classes/assets
